# Trabajo 3: Análisis de datos con NumPy y Pandas (RetailNow)

Este notebook analiza **ventas**, **inventarios** y **satisfacción** por tienda, aplicando:

- Limpieza de datos (`dropna`)
- KPIs con **Pandas**
- Estadística y simulación con **NumPy** (mediana, desviación estándar, proyección futura)

**Archivos CSV (en la raíz del proyecto):**
- `python_intermedio/sales.csv`
- `python_intermedio/inventories.csv`
- `python_intermedio/satisfaction.csv`


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)


## 1) Carga de datos (Pandas) y limpieza básica

> Se intenta cargar desde `python_intermedio/` (ruta del proyecto).  
> Si no existe (por ejemplo, en otros entornos), se usa un *fallback* a rutas típicas.


In [ ]:
# Carga de datos desde rutas absolutas exigidas por el enunciado
from pathlib import Path
import pandas as pd

SALES_PATH = Path("/workspace/sales.csv")
INV_PATH   = Path("/workspace/inventories.csv")
SAT_PATH   = Path("/workspace/satisfaction.csv")

for p in [SALES_PATH, INV_PATH, SAT_PATH]:
    if not p.exists():
        raise FileNotFoundError(f"No se encontró {p}")

sales_raw = pd.read_csv(SALES_PATH)
inv_raw   = pd.read_csv(INV_PATH)
sat_raw   = pd.read_csv(SAT_PATH)

sales = sales_raw.dropna().copy()
inv   = inv_raw.dropna().copy()
sat   = sat_raw.dropna().copy()

print("Datos cargados correctamente")
print("Sales:", sales.shape)
print("Inventories:", inv.shape)
print("Satisfaction:", sat.shape)


## 2) Exploración rápida

In [ ]:
display(sales.head())
display(inv.head())
display(sat.head())

print("\nColumnas sales:", list(sales.columns))
print("Columnas inventories:", list(inv.columns))
print("Columnas satisfaction:", list(sat.columns))


## 3) Preparación de datos: tipos numéricos y columnas derivadas

In [ ]:
# Columnas esperadas (según los CSV proporcionados)
# Ventas
COL_TIENDA = "ID_Tienda"
COL_PRODUCTO = "Producto"
COL_CANT = "Cantidad_Vendida"
COL_PRECIO = "Precio_Unitario"

# Inventario
COL_STOCK = "Stock_Disponible"

# Satisfacción
COL_SAT = "Satisfacción_Promedio"

# Asegurar tipos numéricos
sales[COL_CANT] = pd.to_numeric(sales[COL_CANT], errors="coerce")
sales[COL_PRECIO] = pd.to_numeric(sales[COL_PRECIO], errors="coerce")
inv[COL_STOCK] = pd.to_numeric(inv[COL_STOCK], errors="coerce")
sat[COL_SAT] = pd.to_numeric(sat[COL_SAT], errors="coerce")

# Re-limpieza tras coerción
sales = sales.dropna().copy()
inv = inv.dropna().copy()
sat = sat.dropna().copy()

# Columna derivada: ingresos por línea de venta
sales["Total_Ventas"] = sales[COL_CANT] * sales[COL_PRECIO]

sales.head()


## 4) Análisis de ventas (Pandas)

### 4.1 Ventas totales (unidades) por tienda y producto


In [ ]:
ventas_tienda_producto = (
    sales.groupby([COL_TIENDA, COL_PRODUCTO], as_index=False)[COL_CANT]
    .sum()
    .rename(columns={COL_CANT: "Total_Unidades_Vendidas"})
)

display(ventas_tienda_producto.head(10))


### 4.2 Ingresos totales por tienda

In [ ]:
ingresos_por_tienda = (
    sales.groupby(COL_TIENDA, as_index=False)["Total_Ventas"]
    .sum()
    .rename(columns={"Total_Ventas": "Ingresos_Totales"})
)

display(ingresos_por_tienda.sort_values("Ingresos_Totales", ascending=False))


### 4.3 Resumen estadístico de las ventas

In [ ]:
display(sales["Total_Ventas"].describe())


## 5) Análisis de inventarios (Pandas)

### 5.1 Unir ventas (unidades) con inventario por tienda y producto


In [ ]:
ventas_unidades_tp = (
    sales.groupby([COL_TIENDA, COL_PRODUCTO], as_index=False)[COL_CANT]
    .sum()
    .rename(columns={COL_CANT: "Unidades_Vendidas"})
)

inv_analisis = inv[[COL_TIENDA, COL_PRODUCTO, COL_STOCK]].merge(
    ventas_unidades_tp,
    on=[COL_TIENDA, COL_PRODUCTO],
    how="left"
)

inv_analisis["Unidades_Vendidas"] = inv_analisis["Unidades_Vendidas"].fillna(0)

display(inv_analisis.head(10))


### 5.2 Rotación de inventarios (ventas / stock)

> Rotación = Unidades_Vendidas / Stock_Disponible  
Se evita división por cero.


In [ ]:
inv_analisis["Rotacion_Inventario"] = np.where(
    inv_analisis[COL_STOCK] > 0,
    inv_analisis["Unidades_Vendidas"] / inv_analisis[COL_STOCK],
    np.nan
)

display(inv_analisis.head(10))


### 5.3 Tiendas con inventario crítico (<10% vendidos respecto al stock)

In [ ]:
inventario_critico = inv_analisis[
    (inv_analisis[COL_STOCK] > 0) & (inv_analisis["Rotacion_Inventario"] < 0.10)
].copy()

# Resumen por tienda: cuántos productos están en nivel crítico
critico_por_tienda = (
    inventario_critico.groupby(COL_TIENDA, as_index=False)
    .agg(
        Productos_Criticos=(COL_PRODUCTO, "nunique"),
        Unidades_Vendidas=("Unidades_Vendidas", "sum"),
        Stock_Total=(COL_STOCK, "sum"),
    )
)

critico_por_tienda["%Vendido_Sobre_Stock"] = np.where(
    critico_por_tienda["Stock_Total"] > 0,
    critico_por_tienda["Unidades_Vendidas"] / critico_por_tienda["Stock_Total"],
    np.nan
)

display(critico_por_tienda.sort_values("Productos_Criticos", ascending=False))


## 6) Satisfacción del cliente (Pandas)

### 6.1 Satisfacción media por tienda


In [ ]:
satisfaccion_por_tienda = (
    sat.groupby(COL_TIENDA, as_index=False)[COL_SAT]
    .mean()
    .rename(columns={COL_SAT: "Satisfaccion_Media"})
)

display(satisfaccion_por_tienda.sort_values("Satisfaccion_Media"))


### 6.2 Tiendas con satisfacción baja (<60%)

In [ ]:
tiendas_baja_satisf = satisfaccion_por_tienda[
    satisfaccion_por_tienda["Satisfaccion_Media"] < 60
].copy()

display(tiendas_baja_satisf.sort_values("Satisfaccion_Media"))


### 6.3 Relación ventas vs satisfacción (tabla resumen)

In [ ]:
resumen_tienda = ingresos_por_tienda.merge(
    satisfaccion_por_tienda,
    on=COL_TIENDA,
    how="left"
)

display(resumen_tienda.sort_values("Ingresos_Totales", ascending=False))


## 7) Operaciones con NumPy

Requisito: usar **NumPy** para calcular:

- Mediana de las ventas totales (por tienda)
- Desviación estándar de las ventas (por tienda)


In [ ]:
ventas_array = resumen_tienda["Ingresos_Totales"].to_numpy()

mediana_ventas = np.median(ventas_array)
desv_std_ventas = np.std(ventas_array, ddof=0)  # poblacional

print("Mediana (Ingresos_Totales por tienda):", mediana_ventas)
print("Desv. estándar (Ingresos_Totales por tienda):", desv_std_ventas)


## 8) Simulación de proyecciones de ventas futuras (NumPy)

Simulamos ingresos futuros para cada tienda durante **N meses**, con ruido normal:
- baseline = ingresos actuales
- variación mensual típica (sigma) = 10% (ajustable)
- seed fija para reproducibilidad


In [ ]:
np.random.seed(42)

n_meses = 6
sigma_pct = 0.10  # 10% volatilidad

baseline = resumen_tienda.set_index(COL_TIENDA)["Ingresos_Totales"].to_numpy()

ruido = np.random.normal(loc=0.0, scale=sigma_pct, size=(baseline.shape[0], n_meses))
proyecciones = baseline.reshape(-1, 1) * (1 + ruido)

proy_df = pd.DataFrame(
    proyecciones,
    index=resumen_tienda[COL_TIENDA],
    columns=[f"Mes_{i+1}" for i in range(n_meses)]
)

display(proy_df.head())


### 8.1 Estadísticas básicas de la simulación

In [ ]:
stats_proy = pd.DataFrame({
    COL_TIENDA: proy_df.index,
    "Promedio_Proyectado": proy_df.mean(axis=1),
    "Min_Proyectado": proy_df.min(axis=1),
    "Max_Proyectado": proy_df.max(axis=1),
    "Std_Proyectado": proy_df.std(axis=1, ddof=0),
}).reset_index(drop=True)

display(stats_proy.sort_values("Promedio_Proyectado", ascending=False))


## 9) Conclusiones

- Identifica **tiendas con inventario crítico** para ajustar aprovisionamiento o revisar demanda.
- Identifica **tiendas con baja satisfacción** para priorizar acciones de mejora (servicio, disponibilidad, procesos).
- Usa la **mediana y desviación estándar** para entender dispersión del rendimiento.
- La **simulación** aporta un rango plausible de ingresos futuros para planificación.
